Open the dataset

In [13]:
import pandas as pd 

df = pd.read_json("C:\\Users\\Dell\\OneDrive\\Desktop\\TextBasedAnalysis\\Musical_Instruments_5.json",lines=True)
df.sample(10)


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
6104,A1VW19Y79DC0GF,B0015RIN6U,"Andrew M. Ward ""My Stolen Life""","[2, 2]",My experience with this little tuner has been ...,5,Perfect...,1299369600,"03 6, 2011"
8337,A55YZTYSWPH96,B003VWKPHC,"Mark R. Segal ""bossjimmark""","[0, 0]",I have been in need of a tuner for far too lon...,5,Works great!,1357689600,"01 9, 2013"
2823,A2XSWF2AJSXQZB,B0002H0A3S,bassestkkm,"[0, 0]",Only issue they break easily. But they last a ...,5,Great Bright Tone!,1354838400,"12 7, 2012"
7484,A1ZV3RNQH852U2,B002VA464S,Grenics,"[1, 1]",This microphone in just a few words IS AWESOME...,5,Short and Sweet,1361145600,"02 18, 2013"
2508,A3HKZSCGFU9T2A,B0002GWFEQ,Teagan Mago,"[0, 0]",This is a handy little contraption. I'm using...,4,Good design,1369872000,"05 30, 2013"
7281,AD4QZUGLI9OSK,B002GYWBIM,"Karl Brody ""Me""","[1, 2]",I bought this amp back when i didnt know much ...,2,Practice Amp,1304121600,"04 30, 2011"
9245,A2HWUC980HM1NP,B005J2PXV0,"UncleHammy ""UncleHammy""","[0, 0]",I wanted to try the Clayton Beehold picks to s...,3,Good picks a little to heavy for my style,1372809600,"07 3, 2013"
7523,A205483DF7XVDE,B002ZH9JQC,"StevenEddy ""Audio Pro""","[0, 0]",This thing works great!I play out live in club...,5,What needs to be said?,1325462400,"01 2, 2012"
2436,A1XNCV67JBJVF7,B0002GML4U,Sam Beckett,"[0, 0]","The title says it best, it's a decent basic mi...",4,Decent basic mic cable.,1365033600,"04 4, 2013"
358,AMU3EIC4EOPX4,B0002CZUTM,Sweed,"[0, 0]",Waited two months before reviewing my les paul...,5,very happy with my les paul,1376870400,"08 19, 2013"


Get the Bing Liu lexicon

In [4]:
from sklearn import preprocessing
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize

print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words in opinion lexicon',
      opinion_lexicon.positive()[:10])
print('Examples of negative words in opinion lexicon',
      opinion_lexicon.negative()[:10])


Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\opinion_lexicon.zip.


Create the “Dictionary”

In [5]:
# Let's create a dictionary which we can use for scoring our review text
nltk.download('punkt')
df.rename(columns={"reviewText": "text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}
 
# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
        word_dict[word] = pos_score
      
# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
        word_dict[word] = neg_score


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Function to do the math

In [7]:
def bing_liu_score(text):
    sentiment_score = 0
    bag_of_words = word_tokenize(text.lower())
    for word in bag_of_words:
        if word in word_dict:
            sentiment_score += word_dict[word]
    return sentiment_score  


In [10]:
df['text'].fillna('no review', inplace=True)
df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)


C:\Users\Dell\AppData\Local\Temp\ipykernel_14572\3791062060.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['text'].fillna('no review', inplace=True)


In [11]:
df[['overall',"text", 'Bing_Liu_Score']].head(10)


,overall,text,Bing_Liu_Score
0,4,Contains some interesting stitches.,1
1,5,I'm a fairly experienced knitter of the one-co...,22
2,4,Great book but the index is terrible. Had to w...,0
3,5,I purchased the Kindle edition which is incred...,4
4,5,Very well laid out and very easy to read.\n\nT...,5
5,5,"Beginning her career as a freelance knitter, M...",15
6,5,This is a terrific stitch handbook (and I have...,9
7,4,The book needs to be coil bound. The content i...,1
8,5,I really am enjoying this book! I like the siz...,12
9,5,Just received this book and looked over it cov...,6


In [12]:
df.groupby('overall').agg({'Bing_Liu_Score':'mean'})


,Bing_Liu_Score
overall,
1,-0.255049
2,0.566098
3,1.158796
4,2.028146
5,2.130005
